# ノンパラメトリック法

前章までの仮説検定の多くは**母集団分布**を仮定しており、その多くは正規分布である。  
母集団分布の仮定のもとで統計検定量$T$を導出し、$T$が従う分布を用いて仮説検定を行う。  
これを**パラメトリック法**というが、**ノンパラメトリック法**は正規分布などの母集団分布の仮定を設けない状態で仮説検定を行うものである。  
ノンパラメトリック法の基本的な考え方は、データを構成している観測値を大きさの順に並べて統計量をつくることである。  
母集団がわかっている場合にも、サンプルサイズが小さい時にはノンパラメトリック法が有効とされるケースが多い。  
以下がノンパラメトリック法の代表的な手法である。  
1. ウィルコクソンの順位和検定…2群の差の検定
2. 符号付順位検定、符号検定…対応がある場合の差の検定
3. クラスカル・ウォリス検定…3群以上の差の検定

# ウィルコクソンの順位和検定

帰無仮説、対立仮説は以下のように設定する。  
帰無仮説：2郡$A,B$の成績の分布は同じ  
対立仮説：郡$A$の成績の分布の形は群$B$と同じだが、悪い(-)方にズレている(片側検定)  
**ウィルコクソンの順位和検定**は、はじめに群$A$と群$B$をあわせて、小さい値から順位を与え、その和$W_A$または$W_B$を検定統計量とする。  
これを**順位和**という。帰無仮説が正しいとすると、2郡の順位はランダムになると考えられる。  
全体の順位の組み合わせ数のうち、$W_A$の値以下となる確率を求める。これがウィルコクソン順位和検定の片側$P-$値である

In [1]:
import itertools
import numpy as np
from scipy.special import comb

x = np.array([30., 20., 52., 40., 50., 35.])
x_A = x[:3]
x_B = x[3:]

# 群Aと群Bのデータを合わせて、小さい順に順位を与える(1~)
rank_A = np.argsort(x)[:3]+1
rank_B = np.argsort(x)[3:]+1

# 群ごとに順位の和を計算する
W_A = np.sum(rank_A)
W_B = np.sum(rank_B)

# P値を求める
comb_all = comb(len(x), len(x_A), exact=True)
all_ranks_list = list(itertools.combinations(range(1,len(x_A)+len(x_B)+1), 3))
a_ranks_list = []
for ranks in all_ranks_list:
    ranks_sum = sum(ranks)
    if ranks_sum <= W_A:
        a_ranks_list.append(ranks)
print(f"片側P-値:{len(a_ranks_list)/comb_all}") # この値を有意水準と比較する

片側P-値:0.35


# 並べ替え検定

ウィルコクソンの順位検定と同様に以下の仮説検定を考える。  
帰無仮説：2郡$A,B$の成績の分布は同じ  
対立仮説：群$A$の成績の分布の形は群$B$と同じだが、悪い(-)方にズレている(片側検定)  
数値の平均$\bar{X}_A$または平均$\bar{X}_B$を検定統計量とする点でウィルコクソンの順位検定と異なる。  
2群$A,B$の平均に差があるか否かを考える。

In [3]:
x = np.array([30., 20., 52., 40., 50., 35.])
x_A = x[:3]
x_B = x[3:]

# 郡ごとの平均値を求める
avg_A = np.average(x_A)
avg_B = np.average(x_B)

# P値を求める
all_vals_list = list(itertools.combinations(x, 3))
a_vals_list = []
for vals in all_vals_list:
    avg_vals = sum(vals)/len(x_A)
    if avg_vals <= avg_A:
        a_vals_list.append(vals)

count_comb_all = comb(len(x), len(x_A), exact=True)
print(f"片側P-値:{len(a_vals_list)/count_comb_all}") # この値を有意水準と比較する

片側P-値:0.25


# 符号付き順位検定

対応がある場合の差の検定で使用する。  
ウィルコクソンの順位検定と同様に並べ替えを考えるが、符号に関しても考慮する。  
はじめに絶対値の小さな順に並べる。ただし値が0になったときは、その観測値を除きサンプルサイズ$n$も減らす。  
並べ替えた順に符号付の順位を割り当て、これらのうち正値の合計$T_+$を検定統計量とする。  
帰無仮説、対立仮説は以下のように設定する。  
帰無仮説：分布$D$の中央値が0  
対立仮説：分布$D$の中央値が0以上(片側検定)  
符号付き順位検定を用いるときは、分布$D$が対称であることが条件

In [4]:
X_before = np.array([30., 20., 52., 40., 50., 35., 25.])
X_after = np.array([15., 11., 52., 46., 61., 55., 50.])

# 差分を算出する
D_val = X_after - X_before
print(f"D:{D_val}")

# 値が0の観測値は除いた、符号付きの順位を算出する
D_val_nonzero = D_val[np.nonzero(D_val)]
D_val_nonzero = sorted(D_val_nonzero, key=abs)
print(f"D':{D_val_nonzero}")

rank_unsigned = np.array(range(1, len(D_val_nonzero)+1))
rank_signed = []
for i, item in enumerate(D_val_nonzero):
    if item < 0:
        rank_signed.append(rank_unsigned[i]*(-1))
    else:
        rank_signed.append(rank_unsigned[i])
print(f"D bar':{rank_signed}")

# 正値の合計T+を検定統計量とする
T_plus_val = sum([i for i in rank_signed if i > 0])
print(f"検定統計量T+:{T_plus_val}")

# 検定統計量T+より大きな値をとる確率を求める
check_counter = 2**len(rank_signed)
tplus_greater_list = []
for counter in range(check_counter):
    # 2進数を用いて組み合わせ計算を実行
    bin_str = format(counter, 'b').zfill(len(rank_signed))
    tmp_rank_signed = []
    tmp_tplus_val = 0
    for i, bin_char in enumerate(bin_str):
        if bin_char=="0":
            tmp_rank_signed.append(rank_unsigned[i]*(-1))
        else:
            tmp_rank_signed.append(rank_unsigned[i])
    tmp_tplus_val = sum([i for i in tmp_rank_signed if i > 0])

    if tmp_tplus_val >=T_plus_val:
        tplus_greater_list.append(tmp_tplus_val)

print(f"片側P-値:{len(tplus_greater_list)/check_counter}") # この値を有意水準と比較する

# scipyを使用して符号付き順位検定を実行することもできる
from scipy import stats

A = np.array([1.83, 1.50, 1.62, 2.48, 1.68, 1.88, 1.55, 3.06, 1.30])
B = np.array([0.88, 0.65, 0.60, 1.05, 1.06, 1.29, 1.06, 2.14, 1.29])
result = stats.wilcoxon(A, B)
print(f"(stats.wilcoxon)検定統計量T+:{result.statistic}, P-値:{result.pvalue}")

D:[-15.  -9.   0.   6.  11.  20.  25.]
D':[6.0, -9.0, 11.0, -15.0, 20.0, 25.0]
D bar':[1, -2, 3, -4, 5, 6]
検定統計量T+:15
片側P-値:0.21875
(stats.wilcoxon)検定統計量T+:0.0, P-値:0.00390625


# 符号検定

帰無仮説、対立仮説は以下のように設定する。  
帰無仮説：分布$D$の中央値が0  
対立仮説：分布$D$の中央値が0以上(片側検定)  
$D$分布の対称性は仮定しなくても問題ない。  
0の値をとらないサンプルサイズ$n$に対して、差$d$の値が正になった個数$T_+$を検定統計量とする。  
帰無仮説が正しいとき、検定統計量$T_+$は二項分布$Bin(n, 0.5)$に従うと考えて$P$値を算出する。

In [8]:
X_before = np.array([30., 20., 52., 40., 50., 35., 25.])
X_after = np.array([15., 11., 52., 46., 61., 55., 50.])
D_val = X_after - X_before
T_plus_val = np.count_nonzero(D_val > 0)    # 正になった個数を検定統計量T+とする
non_zero_num = np.count_nonzero(D_val != 0)
print(f"検定統計量T+:{T_plus_val}")

# Bin(6,0.5)に従うと考えてP値を算出
result = stats.binomtest(k=T_plus_val, n=non_zero_num, p=0.5, alternative='greater')
result

検定統計量T+:4


BinomTestResult(k=4, n=6, alternative='greater', statistic=0.6666666666666666, pvalue=0.34375)

# クラスカル・ウォリス検定

**クラスカル・ウォリス検定**は、複数の群の分布に差があるか否かを考える。  
帰無仮説、対立仮説は以下のように設定する。  
帰無仮説：全ての郡の分布は同じ  
対立仮説：全ての郡の分布は同じではない  
ウィルコクソンの順位和検定と同様に、各群の分布が同じであることが前提である。  
また、すべての群のうち、どの群がどの群と異なるかどうかということは考慮しない。

In [9]:
# scipyを使用して算出することができる
group_a = np.array([22., 30., 42.])
group_b = np.array([36., 40., 52., 53.])
group_c = np.array([25., 32., 45., 48.])

result = stats.mstats.kruskalwallis(group_a, group_b, group_c)
print(f"検定統計量H:{result.statistic:.3f}, P-値:{result.pvalue:.3f}")

検定統計量H:2.962, P-値:0.227


# 順位相関係数

2次元データが共に順位データである場合の相関係数のことを**順位相関係数**という。  
**スピアマンの順位相関係数**と**ケンドールの順位相関係数**がある。  
ともに-1から1の間をとるが、同じ値にはならないため互いの意味は異なる。

スピアマンの順位相関係数は、2次元データ$(x_i,y_i)$がともに連続変数である場合のピアソンの積率相関係数と同じ計算を行う

In [10]:
# スピアマンの順位相関係数を計算
x_val = np.array([1., 2., 3., 4., 5., 6., 7.])
y_val = np.array([1., 3., 2., 6., 4., 5., 7.])
diff = x_val - y_val
square_diff_val = np.square(diff).sum()
n = len(x_val)

rs_val = 1 - (6*square_diff_val / (n*(n**(2)-1)) )
print(f"スピアマンの順位相関係数:{rs_val:.3f}")

# scipyのAPIを使用して算出した場合
corr, pvalue = stats.spearmanr(x_val, y_val)
print(f"(stats.spearmanr)スピアマンの順位相関係数:{corr:.3f}")

スピアマンの順位相関係数:0.857
(stats.spearmanr)スピアマンの順位相関係数:0.857


ケンドールの順位相関係数は、$x_i$と$x_j$、$y_i$と$y_j$の順位の大きさの順が一致している組の数と、逆のある組の数の差をみている。

In [11]:
# ケンドールの順位相関係数を計算
x_val = np.array([1., 2., 3., 4., 5., 6., 7.])
y_val = np.array([1., 3., 2., 6., 4., 5., 7.])
P_count = 0
N_count = 0
n = len(x_val)

for i in range(len(x_val)):
    for j in range(len(x_val)):
        if i >= j:
            continue
        val = (x_val[i]-x_val[j])*(y_val[i]-y_val[j])
        if val > 0:
            P_count+=1
        else:
            N_count+=1

rk_val = (P_count - N_count) / (n*(n-1)/2)
print(f"ケンドールの順位相関係数:{rk_val:.3f}")

# scipyのAPIを使用して算出した場合
corr, pvalue = stats.kendalltau(x_val, y_val)
print(f"(stats.kendalltau)ケンドールの順位相関係数:{corr:.3f}")

ケンドールの順位相関係数:0.714
(stats.kendalltau)ケンドールの順位相関係数:0.714
